In [1]:
import os
import matplotlib.pyplot as plt
from matplotlib.ticker import StrMethodFormatter
import pandas as pd
import numpy as np
import fiona
import seaborn as sns
import geopandas as gpd
import earthpy as et
import earthpy.plot as ep

In [2]:
def download_import_gdf(file_url, path):   
    """Download file from specified URL and import file
    into a geopandas dataframe from a specified path. If
    URL does not work or exist create geopandas dataframe 
    from manual download with a path.
    
    Working directory is set to 'earth-analytics', 'data' 
    directory under home, which is automatically created
    by the download. 

    
    Parameters
    ----------
    file_url : str
        URL to file (http or https).
     path : str
        Path to file using relative path
        to 'earth-analytics', 'data' directory under home.   

    Returns
    ------
    gdf : geopandas dataframe
        Dataframe imported from downloaded file.
    """ 
    if file_url.startswith("http"):
        try:
            et.data.get_data(url=file_url)      
            os.chdir(os.path.join(et.io.HOME, "earth-analytics", "data"))
            gdb_file = path
            layers = fiona.listlayers(gdb_file)
            for layer in layers:
                gdf = gpd.read_file(gdb_file,layer=layer, rows = 5)
                #can remove the argument rows=5, left in for quicker processing
            return gdf
        except Exception as err:
            print(err)
            return None
    else:
        print ("Download manually")
        os.chdir(os.path.join(et.io.HOME, "earth-analytics", "data")) 
        gdf = gpd.read_file(path)
    return gdf

In [3]:
ffr_url = "https://ndownloader.figshare.com/files/15090536"
ffr_path = os.path.join("earthpy-downloads",
                        "Mapping%20the%20worlds%20free-flowing%20rivers_Data_Geodatabase",
                        "Mapping the worlds free-flowing rivers_Data_Geodatabase",
                        "FFR_river_network.gdb")

In [4]:
# Create dataframe using download/import function
ffr_gdf = download_import_gdf(
    file_url=ffr_url,
    path=ffr_path)

ffr_gdf

,REACH_ID,GOID,NOID,NUOID,NDOID,CON_ID,CONTINENT,COUNTRY,BAS_ID,BAS_NAME,...,RDD,FLD,CSI,CSI_D,CSI_FF,CSI_FF1,CSI_FF2,CSI_FFID,Shape_Length,geometry
0,10000001.0,1,1,,0,4,Africa,Tunisia,2609115,,...,0.000,19.697001,99.28000,USE,1,1,1,1,0.008839,"MULTILINESTRING ((9.66250 37.32500, 9.65625 37..."
1,10000002.0,2,2,,0,4,Africa,Tunisia,2609123,,...,1.083,9.722000,99.34585,USE,1,1,1,2,0.028957,"MULTILINESTRING ((9.80000 37.30833, 9.81042 37..."
2,10000003.0,3,3,,9,4,Africa,Tunisia,2609811,,...,0.475,17.910000,98.92625,USE,1,1,1,9,0.046487,"MULTILINESTRING ((9.68542 37.27083, 9.68542 37..."
3,10000004.0,4,4,,9,4,Africa,Tunisia,2609811,,...,2.429,15.385000,99.38355,USE,1,1,1,10,0.006250,"MULTILINESTRING ((9.71458 37.23750, 9.71458 37..."
4,10000005.0,5,5,,0,4,Africa,Tunisia,2609655,,...,2.167,20.488001,98.35100,USE,1,1,1,3,0.085713,"MULTILINESTRING ((9.75000 37.27708, 9.75625 37..."


In [5]:
grand_url = "https://ln.sync.com/dl/bd47eb6b0/anhxaikr-62pmrgtq-k44xf84f-pyz4atkm"
grand_path = os.path.join("earthpy-downloads", "GRanD_Version_1_3", 
                          "GRanD_dams_v1_3.shp")

In [6]:
# Create dataframe using download/import function
grand_gdf = download_import_gdf(
    file_url=grand_url,
    path=grand_path)

grand_gdf

,GRAND_ID,RES_NAME,DAM_NAME,ALT_NAME,RIVER,ALT_RIVER,MAIN_BASIN,SUB_BASIN,NEAR_CITY,ALT_CITY,...,MULTI_DAMS,TIMELINE,COMMENTS,URL,QUALITY,EDITOR,LONG_DD,LAT_DD,POLY_SRC,geometry
0,1,None,Terror Lake,None,Terror River,Marmont Bay,None,None,None,None,...,None,None,None,http://www.ferc.gov/industries/hydropower/safe...,3: Fair,UNH,-153.026649,57.651485,NHD,POINT (-153.02665 57.65149)
1,2,None,Mayo,None,Mayo,None,None,None,Whitehorse,None,...,None,None,This dam is on Mayo Lake to control the releas...,http://www.yukon-news.com/opinions/columns/12540/,1: Verified,McGill-BL,-135.367048,63.774184,CanVec,POINT (-135.36705 63.77418)
2,3,None,Blue Lake,None,Sawmill Creek,None,None,None,None,None,...,None,None,None,http://www.ferc.gov/industries/hydropower/safe...,2: Good,UNH,-135.200305,57.062136,NHD,POINT (-135.20030 57.06214)
3,4,None,Green Lake,None,Vodopad River,None,None,None,None,None,...,None,None,None,http://www.ferc.gov/industries/hydropower/safe...,2: Good,UNH,-135.112812,56.986785,NHD,POINT (-135.11281 56.98679)
4,5,Long Lake,Long Lake Dam,Snettisham Dam,Long River,None,None,None,None,None,...,None,None,None,http://www.dnr.state.ak.us/mlw/water/dams/,4: Poor,UNH,-133.728413,58.168750,NHD,POINT (-133.72841 58.16875)


In [9]:
# Are both datasets in the same CRS?
if (grand_gdf.crs == ffr_gdf.crs):
    print("Both layers are in the same crs!",
          grand_gdf.crs, ffr_gdf.crs)
else:
    print ("Reproject data to EPSG: 4326")

Both layers are in the same crs! epsg:4326 epsg:4326


In [8]:
grand_gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich